In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from pymongo import MongoClient
import time
import re

In [2]:
# Kết nối MongoDB
client = MongoClient("mongodb://localhost:27017/")
client.drop_database('pharmacity')
db = client['pharmacity']

In [3]:

products_collection = db['products']
sales_collection = db['sales']


In [4]:

# Khởi tạo WebDriver cho Firefox
driver = webdriver.Chrome()

# Truy cập vào trang dược phẩm
driver.get("https://www.pharmacity.vn/duoc-pham")

# Đợi trang tải
time.sleep(3)

In [5]:
# Hàm cào dữ liệu từng sản phẩm
def scrape_product(product_link):
    # Mở link sản phẩm
    driver.get(product_link)
    time.sleep(2)

    # Lấy mã sản phẩm
    try:
        product_code = driver.find_element(By.CSS_SELECTOR, "p.text-sm.leading-5.text-neutral-600").text
    except:
        product_code = "N/A"

    # Lấy tên sản phẩm
    try:
        product_name = driver.find_element(By.CSS_SELECTOR, "h1.text-neutral-900.font-semibold").text
    except:
        product_name = "N/A"

    try:
        product_img = driver.find_element(By.XPATH,
                                          '//*[@id="mainContent"]/div/div[1]/div[3]/div[1]/div[1]/div[1]/div/div[1]/div/div/div[1]/div/img').get_attribute(
            'src')
    except:
        product_img = "N/A"

    # Lấy thương hiệu
    try:
        product_brand = driver.find_element(By.CSS_SELECTOR,
                                            '#mainContent > div > div:nth-child(1) > div.relative.grid.grid-cols-1.gap-6.md\\:container.md\\:grid-cols-\\[min\\(60\\%\\,calc\\(555rem\\/16\\)\\)\\,1fr\\].md\\:pt-6.lg\\:grid-cols-\\[min\\(72\\%\\,calc\\(888rem\\/16\\)\\)\\,1fr\\] > div.grid.md\\:gap-6 > div.grid.grid-cols-1.items-start.md\\:gap-6.lg\\:grid-cols-2.xl\\:grid-cols-2 > div:nth-child(2) > div > div.flex.flex-col.px-4.md\\:px-0 > div.gap-3.md\\:gap-4.mb-3.grid.md\\:mb-4 > div.grid.gap-3.md\\:gap-2 > div:nth-child(5) > div').text
    except:
        product_brand = "N/A"

    # Lấy nơi sản xuất
    try:
        product_origin = driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/div[3]/div[2]/div/a').text
    except:
        product_origin = "N/A"

    # Lấy giá bán
    try:
        product_price = driver.find_element(By.TAG_NAME, 'h3').text
        # Sử dụng regex để loại bỏ các ký tự không phải số và dấu phân cách thập phân
        # Giữ lại số và dấu chấm (.)
        cleaned_price_str = re.sub(r'[^\d.]', '', product_price)
        product_price = float(cleaned_price_str) * 1000
    except:
        product_price = "N/A"

    # Lấy lượt yêu thích (nếu có)
    try:
        product_likes = driver.find_element(By.CSS_SELECTOR, 'div.space-x-1:nth-child(2) > p:nth-child(1)').text
        cleaned_likes_str = re.sub(r'[^\d.]', '', product_likes)
        product_likes = float(cleaned_likes_str) * 1000
    except:
        product_likes = "N/A"

    # Lấy số lượng bán (nếu có)
    try:
        product_sold = driver.find_element(By.CSS_SELECTOR, 'p.text-sm:nth-child(3)').text
        cleaned_sold_str = re.sub(r'[^\d.]', '', product_sold)
        product_sold = float(cleaned_sold_str) * 1000
    except:
        product_sold = "N/A"

    try:
        product_type = driver.find_element(By.CSS_SELECTOR, "div.md\\:text-base").text
    except:
        product_type = "N/A"

    # Tạo từ điển lưu thông tin sản phẩm
    product_data = {
        "Product_ID": product_code,
        "Product_Name": product_name,
        "Type": product_type,
        "Img": product_img,
        "Brand": product_brand,
        "Prpduct_origin": product_origin,
        "Price": product_price,
        "Link": product_link
    }
    sale_data = {
        "Product_ID": product_code,
        "Product_Name": product_name,
        "Likes": product_likes,
        "Sold": product_sold
    }

    # Lưu vào MongoDB
    products_collection.insert_one(product_data)
    sales_collection.insert_one(sale_data)
    print(f"Đã lưu: {product_name}")


# # Hàm nhấn nút "Xem thêm" để tải thêm sản phẩm
# def load_more_products():
#     try:
#         load_more_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Xem thêm')]")
#         load_more_button.click()
#         time.sleep(3)  # Đợi trang tải thêm sản phẩm
#     except:
#         print("Không còn sản phẩm để tải thêm.")
#
# # Hàm cuộn trang xuống cuối để tải thêm sản phẩm
# def scroll_down():
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(2)


# Lấy danh sách link sản phẩm
def get_product_links():
    product_links = []
    products = driver.find_elements(By.CSS_SELECTOR, "a:has(h3.line-clamp-2.h-10.text-sm.font-semibold)")
    for product in products:
        product_link = product.get_attribute("href")
        product_links.append(product_link)
    return product_links


# Cào dữ liệu từ trang web
while True:
    try:
        # Lấy danh sách link sản phẩm trên trang
        links = get_product_links()

        # Cào dữ liệu từng sản phẩm
        for link in links:
            scrape_product(link)
    except:
        print("Đã lưu hết dược phẩm!")
        break

Đã lưu: Acritel 10g (Hộp 6 vỉ x 10 viên)
Đã lưu: AtiSalbu 2mg (30 ống x 5ml/hộp)
Đã lưu: N/A
Đã lưu hết dược phẩm!


## đóng driver

In [6]:
driver.quit()

In [7]:
client = MongoClient("mongodb://localhost:27017/")
db = client['pharmacityDB']
products_collection = db['products']
sales_collection = db['sales']
products_detail = db['details']

In [8]:

# 1.Hiện tất cả các sản phẩm và số lượng sản phẩm trong collection 'products'
for product in db.products.find():
    print(product)
sl = db.products.count_documents({})
print(f'Tổng số sản phẩm trong products là: {sl}')
print('\n')

Tổng số sản phẩm trong products là: 0




In [9]:
# 2.Tìm sản phẩm không kê đơn có giá cao nhất
highest_price = db.products.find({"Type": "Thuốc không kê đơn", "Price": {"$ne": "N/A"}}).sort("Price", -1).limit(1)
print("Thuốc không kê đơn có giá cao nhất:")
for product in highest_price:
    print(product)
print('\n')

Thuốc không kê đơn có giá cao nhất:




In [10]:
# 3.Tìm sản phẩm không kê đơn có giá thấp nhất
lowest_price = db.products.find({"Type": "Thuốc không kê đơn"}).sort("Price", 1).limit(1)
print("Thuốc không kê đơn có giá thấp nhất:")
for product in lowest_price:
    print(product)
print('\n')

Thuốc không kê đơn có giá thấp nhất:




In [11]:
# 4.Lấy sản phẩm có thành phần hoạt tính chứa "Levocetirizin"
timhoattinh = db.details.find({"Active_element": {"$regex": "Levocetirizin", "$options": "i"}})
for tim in timhoattinh:
    print(f'Thuốc có hoạt tính Levocetirizin là {tim}')
print('\n')

In [12]:
# 5.Đếm số sản phẩm có nguồn gốc từ "Việt Nam"
fromVN = db.details.count_documents({"Product_origin": "Việt Nam"})
print(f'Tổn số sản phẩm đến từ VN là {fromVN}')
print('\n')

Tổn số sản phẩm đến từ VN là 0




In [13]:
# 6.Đếm số sản phẩm không có nguồn gốc từ "Việt Nam"
notfromVN = db.details.count_documents({"Product_origin": {"$ne":"Việt Nam"}})
print(f'Tổn số sản phẩm không đến từ VN là {notfromVN}')
print('\n')

Tổn số sản phẩm không đến từ VN là 0




In [14]:
# 7.Tìm sản phẩm có giá bán hơn 100k
print('Sản phẩm có giá bán hơn 100k:')
for p in db.products.find({"Price": {"$gt": 100000}}):
    print(p)
print('\n')

Sản phẩm có giá bán hơn 100k:




In [15]:
# 8.Tìm sản phẩm có số lượng bán hơn 5000
print('Sản phẩm có số lượng bán hơn 5000:')
for p in db.sales.find({"Sold": {"$gt": 5000}}):
    print(p)
print('\n')

Sản phẩm có số lượng bán hơn 5000:




In [16]:
# 9.Lấy thông tin chi tiết sản phẩm và với thông tin bán hàng
product_sales_details = db.sales.aggregate([
    {
        "$lookup": {
            "from": "products",
            "localField": "Product_ID",
            "foreignField": "Product_ID",
            "as": "product_info"
        }
    }
])
print('Thông tin chi tiết sản phẩm: ')
for p in product_sales_details:
    print(p)
print('\n')

Thông tin chi tiết sản phẩm: 




In [17]:
# 10.Tìm sản phẩm có tên chứa từ khóa Eagle
print('Sản phẩm có tên chứa từ khóa Eagle: ')
for p in db.products.find({"Product_Name": {"$regex": "Eagle"}}):
    print(p)
print('\n')

Sản phẩm có tên chứa từ khóa Eagle: 




In [18]:
# 11. Tìm sản phẩm theo Product_ID
product = db.products.find_one({"Product_ID": "P14941"})
print(f'Sản phẩm có ID P14941 là: \t{product}')
print('\n')

Sản phẩm có ID P14941 là: 	None




In [19]:
# 12.Tìm sản phẩm có số lượt thích thấp nhất và trả về tên, giá bán, like só lượng bán ra sản phẩm
lowest = db.sales.aggregate([{
        "$lookup": {
            "from": "products",
            "localField": "Product_ID",
            "foreignField": "Product_ID",
            "as": "product_info"
        }},
    {
        "$unwind": "$product_info"},{"$sort": {"Likes": 1}},{"$limit": 1},
    {
        "$project": {
            "Product_Name": "$product_info.Product_Name",
            "Price": "$product_info.Price",
            "Likes": "$Likes",
            "Sold": "$Sold"
        }
    }
])
print('Sản phẩm có lượt thích thấp nhất là:')
for product in lowest:
    print(product)
print('\n')

Sản phẩm có lượt thích thấp nhất là:




In [20]:
# 13.Tìm sản phẩm có số lượt thích cao nhất và trả về tên, giá bán, like só lượng bán ra sản phẩm
highest = db.sales.aggregate([
    {
        "$lookup": {
            "from": "products",
            "localField": "Product_ID",
            "foreignField": "Product_ID",
            "as": "product_info"}},
    {"$unwind": "$product_info"},
    {"$match": {"Likes": {"$ne": "N/A"}} },
    {"$sort": {"Likes": -1}},
    {"$limit": 1  },
    {"$project": {
            "Product_Name": "$product_info.Product_Name",
            "Price": "$product_info.Price",
            "Likes": "$Likes",
            "Sold": "$Sold"
        }}
])
print('Sản phẩm có lượt thích cao nhất là:')
for product in highest:
    print(product)
print('\n')

Sản phẩm có lượt thích cao nhất là:




In [21]:
# 14.Tính tổng số lượng sản phẩm bán được từ collection 'sales'
sold_c = db.sales.aggregate([{"$group": {"_id": None,
                                         "Tổng số thuốc không kê đơn bán ra là:": {"$sum": "$Sold"}}}])
for c in sold_c:
    print(c)
print('\n')

In [22]:
# 15.Tính tổng số tiền bán thuốc không kê đơn
total_sales = db.sales.aggregate([
    {
        "$lookup": {
            "from": "products",
            "localField": "Product_ID",
            "foreignField": "Product_ID",
            "as": "product_info"}},
    {"$unwind": "$product_info"},
    {"$match": {"product_info.Type": "Thuốc không kê đơn", "product_info.Price": {"$ne": "N/A"}},},
    {"$group": {
            "_id": None,
            "Tổng số tiền bán thuốc không kê đơn là": {"$sum":
                                                           {"$multiply": ["$Sold", "$product_info.Price"]}}
        }}
])
for s in total_sales:
    print(s)

## đóng close dbs

In [23]:
client.close()